<a href="https://colab.research.google.com/github/sayan112207/Perplexa/blob/main/Perplexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Dec 17 07:40:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#Scrape Search Enginge

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import re

In [ ]:
def get_vqd(query: str):
    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "accept-language": "en-TH,en;q=0.9",
        "priority": "u=0, i",
        "referer": "https://duckduckgo.com/",
        "sec-ch-ua": '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"macOS"',
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
    }

    params = {
        "q": query,
        "t": "h_",
    }

    response = requests.get("https://duckduckgo.com/", params=params, headers=headers)

    html = response.text

    found = re.search('vqd="(([0-9]|-)*)"', html, re.IGNORECASE)
    if found:
        vqd = found.group(0).replace("vqd=", "").strip('"')
        return vqd

In [ ]:
def get_data(q: str, vqd: str, language: str = "th-th") -> list:
    params = {
        "q": q,
        "l": language,
        "s": "0",
        "a": "h_",
        "dl": "th",
        "ct": "TH",
        "vqd": vqd,
    }
    response = requests.get("https://links.duckduckgo.com/d.js", params=params)

    return json.loads(
        response.text.split("DDG.pageLayout.load('d',")[-1].split(");")[0]
    )

In [ ]:
def clean_data(raw_search_result: dict) -> dict:
    header = raw_search_result.get("a")
    sample_text = raw_search_result.get("t")
    url = raw_search_result.get("c")
    post_date = raw_search_result.get("e")

    return {
        "header": header,
        "sample_text": sample_text,
        "url": url,
        "post_date": post_date,
    }

In [ ]:
def search(q: str) -> list:
    vqd = get_vqd(q)

    raw_search_results = get_data(q, vqd)
    cleaned_results = [
        clean_data(raw_search_result) for raw_search_result in raw_search_results
    ]
    return cleaned_results

In [ ]:
search_results = search("What is Javascript")
print(search_results)

[{'header': '<b>JavaScript</b> <b>is</b> a scripting language that allows you to create dynamic and interactive web pages. Learn what <b>JavaScript</b> can do, how it fits into the web technologies stack, and how to use it with examples and demos.', 'sample_text': 'What is JavaScript? - Learn web development | MDN - MDN Web Docs', 'url': 'https://developer.mozilla.org/en-US/docs/Learn/JavaScript/First_steps/What_is_JavaScript', 'post_date': None}, {'header': 'Learn the basics of <b>JavaScript</b>, a single-threaded, synchronous programming and scripting language widely used in web development. Find out its history, applications, advantages, limitations, and how to include it in HTML.', 'sample_text': 'What is JavaScript? - GeeksforGeeks', 'url': 'https://www.geeksforgeeks.org/what-is-javascript/', 'post_date': None}, {'header': '<b>JavaScript</b> <b>is</b> a programming language and core technology of the Web, alongside HTML and CSS. Learn about its creation at Netscape, adoption by Mi

#Implementing RAG Architecture

##Installing Ollama

In [ ]:
!sudo apt-get install -y pciutils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install ollama retry

##Setting Ollama host

In [ ]:
from IPython.display import clear_output
import os
import threading
import subprocess
import requests
import json

In [ ]:
def ollama():
    try:
        os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
        os.environ['OLLAMA_ORIGINS'] = '*'
        subprocess.Popen(["ollama", "serve"])
        print("Ollama server started successfully.")
    except Exception as e:
        print("Error starting Ollama server:", e)

In [ ]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
!ollama --version

ollama version is 0.5.1


##Downloading Text-Generation Model

In [ ]:
!ollama pull qwen2:7b-instruct-q6_K
clear_output()

In [ ]:
!ollama list

NAME                      ID              SIZE      MODIFIED       
qwen2:7b-instruct-q6_K    43f1d6679e17    6.3 GB    21 minutes ago    


In [ ]:
import ollama

MODEL = "qwen2:7b-instruct-q6_K"
OLLAMA_HOST = "http://localhost:11434"
RETRY_COUNT = 5
SELECT_TOP_RESULTS = 3
INDEX_NAME = "sfc_code_preprocess"
ollama_client: ollama.Client = ollama.Client(host=OLLAMA_HOST)

In [ ]:
from retry import retry
from typing import Union, Generator
from IPython.display import clear_output, Markdown

##Implemting RAG

In [ ]:
def get_search_keywords(question):
    response = ollama_client.chat(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "You are an SEO expert. A user will give you a question your duty is to figure it out the search keywords for using in a search engine"
                "such as DuckDuckGo to gather the information. "
                "You must return your answer only  and do not include prologue, prefix or suffix.",
            },
            {"role": "user", "content": question},
        ],
        stream=False,
    )
    return response["message"]["content"].split()

In [ ]:
def summarize_document(question, document):
    prompt = f"""Question: {question}

Document: {document}

Summarize context that can directly answer the question into bullets point to write a content.
You must give your answer in Thai language and return your answer only  and do not include prologue, prefix or suffix.

"""
    response = ollama_client.chat(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "You are an research assistance who summarizes a"
                " given a document scraped from a webpage to answer a given question."
                " You must give your answer in Thai language and do not include prologue, prefix or suffix.",
            },
            {"role": "user", "content": prompt},
        ],
        stream=False,
    )
    return response["message"]["content"]

In [ ]:
def get_is_useful_prompt(question, context) -> str:
    prompt = f"""From the question: {question}
                  And context {context}

                  Consider if the context is what the question want or not. You must answer with -1 if it's not;
                  0 if it's partially related; 1 it it's related. You must give the answer only and do not include prologue, prefix or suffix."""
    return prompt

In [ ]:
def get_summary_prompt(question, document):
    prompt = f"""Question: {question}
                  Document: {document}

                  Prepare information that can to answer the question into bullets point to write a content.
                  Also, consider if the docuement is useful to answer the question.
                  Your answer should be in JSON format with fields:

                  "is_useful": 0 if not or 1 if it's useful
                  "summarize": (string) Summarize of the that can response the question or leave blank if you find it entirely not useful.


                  Return your answer only and do not include prologue, prefix or suffix."""
    return prompt

In [ ]:
@retry(tries=5, exceptions=json.JSONDecodeError)
def validate_document(question, document):
    summary_prompt = get_summary_prompt(question, document)
    response = ollama_client.chat(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": (
                    """You are an expert who is assigned to consider whether a text data source
                    is useful to answer a user question or not. If yes, you will summarize the text
                    which corespond user's question for another expert to write answer the user , otherwise, do nothing."""

                    """You answer must be in JSON format with field:
                    "is_useful": boolean determining whether the source is useful,
                    "summarize: string your summarization refering the part for the text or empty string if not useful
                    """

                    "return your answer only  and do not include prologue, prefix or suffix"
                ),
            },
            {"role": "user", "content": summary_prompt},
        ],
        stream=False,
    )
    return json.loads(response["message"]["content"])

In [ ]:
def get_context(question):
    contexts = []

    search_results = search(question)
    for search_result in search_results:

        document = search_result["header"]
        context = validate_document(question, document)


        is_useful = context["is_useful"]
        search_result["is_useful"] = is_useful
        search_result["summarize"] = context["summarize"]
        if is_useful == 1:
            contexts.append(search_result)
        if len(contexts) >= 5:
            break

    line_template = """Header:
    {header}

    Summary:
    {summarize}

    Reference: {url}
    """

    writing_prompt = "\n".join([line_template.format(**context) for context in contexts])
    return writing_prompt

###Model Response [Object: ChatResponse]

In [ ]:
def write_the_output(question, context_prompts):
    response = ollama_client.chat(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful user's assistant, and your secretary already "
                    "prepared gists from the search engine results for "
                    "you to answer user's question. "
                    "Your duty is to answer the question "
                    "with confidence using the prepared "
                    "data source as a reference. "
                    "You must add your idea to support user to understand sources. "
                    "You must add the reference of data source with URL "
                    "and encourage user to find out more information with it. "
                    "You must answer with the same language that the user uses."
                ),
            },
            {
                "role": "user", "content": f"""Question: {question}
                Context: {context_prompts}
                """
            },
        ],
        stream=False,
    )
    return response

###Assessing only Content attribute of ChatResponse Object

In [ ]:
def display_answer(answer):
    content = answer.message.content
    return content

In [ ]:
def suplexity(question):
    context_prompts = get_context(question)
    stream = write_the_output(question, context_prompts)
    return stream

#Final Display

In [ ]:
print(display_answer(suplexity("What is JavaScript")))

JavaScript is a scripting language that helps in making websites more interactive. It allows developers to create dynamic elements like forms, animations, and interactive user interfaces directly within the web pages without requiring a full page refresh. JavaScript can be used both on the client-side (in users' browsers) and the server-side (on servers).

Here are some key points about JavaScript:

1. **Web Technologies Stack**: JavaScript is an integral part of the web technologies stack alongside HTML (HyperText Markup Language) for structuring content, and CSS (Cascading Style Sheets) for styling it.

2. **Broad Applications**: JavaScript can be used to perform a wide range of tasks on websites like form validation, creating interactive pop-ups, animations, data manipulation, etc., enhancing user experience significantly.

3. **Versatility**: It is not just limited to web applications; with frameworks and libraries such as Node.js, it's also widely used in server-side development.
